<a href="https://colab.research.google.com/github/Maneshgpai/chatbot/blob/main/medcode_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install llama-index streamlit

In [ ]:
!npm install localtunnel

In [25]:
%%writefile app.py
import os
import streamlit as st


DEFAULT_TERM_STR = (
    "Make a list of terms and definitions that are defined in the context, "
    "with one pair on each line. "
    "If a term is missing it's definition, use your best judgment. "
    "Write each line as as follows:\nTerm: <term> Definition: <definition>"
)

st.title("🦙 Llama Index Term Extractor 🦙")

setup_tab, upload_tab = st.tabs(["Setup", "Upload/Extract Terms"])

with setup_tab:
    st.subheader("LLM Setup")
    api_key = st.text_input("Enter your OpenAI API key here", type="password")
    llm_name = st.selectbox('Which LLM?', ["text-davinci-003", "gpt-3.5-turbo", "gpt-4"])
    model_temperature = st.slider("LLM Temperature", min_value=0.0, max_value=1.0, step=0.1)
    term_extract_str = st.text_area("The query to extract terms and definitions with.", value=DEFAULT_TERM_STR)

with upload_tab:
    st.subheader("Extract and Query Definitions")
    document_text = st.text_area("Or enter raw text")
    if st.button("Extract Terms and Definitions") and document_text:
        with st.spinner("Extracting..."):
            extracted_terms = ""  # this is a placeholder!
        st.write(extracted_terms)

Writing app.py


In [26]:
!streamlit run app.py &>/content/logs.txt &

In [29]:
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.204.4.60
npx: installed 22 in 2.667s
your url is: https://rude-cities-occur.loca.lt


_____________________________________________________________________

In [ ]:
pip install langchain pypdf openai

In [9]:
import openai
import requests
import json
import os
import time
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader

In [4]:
os.environ['OPENAI_API_KEY'] = "sk-...5z7h"
openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
def get_codes(query):
    prompt = f"Given the following doctor's dictation transcript, \
    provide the relevant ICD-10 and CPT codes and their descriptions:\n\n{query}\n\n\
    You can find an updated list of ICD codes at this site: https://www.icd10data.com/ICD10CM/Codes. \
    ICD-10 and CPT Codes:"
    response = openai.Completion.create(
        engine="text-davinci-002", prompt=prompt, max_tokens=1000, n=1, stop=None, temperature=0,
    )

    codes_text = response.choices[0].text.strip()
    codes_table = codes_text.split("\n")

    return codes_table

In [ ]:
## Function to load file
def load_file(file_path):
  documents = []
  split_tup = os.path.splitext(file_path)
  print(split_tup)

  # extract the file name and extension
  file_extension = split_tup[1]
  print("File Extension: ", file_extension)

  if file_extension.casefold() == 'pdf':
    loader = PyPDFLoader(file_path)
  elif file_extension.casefold() == 'txt':
    loader = TextLoader(file_path)
  else:
    raise Exception("Format not recognized!!!")

  documents = loader.load()
  return documents

In [ ]:
## Function to get transcript from audio file
def mp3_to_text(file_path):
    # Upload the audio file
    with open(file_path, "rb") as f:
        response = openai.Audio.transcribe("whisper-1", file=f)

    transcript = response['text']
    return transcript

In [ ]:
## Function to retract PII from PDF. Use either Google API or Llamaindex
## https://cloud.google.com/dlp/docs/redacting-sensitive-data-images?hl=en
## https://gpt-index.readthedocs.io/en/latest/examples/node_postprocessor/PII.html

# def retract_pii()

In [ ]:
def main():
    query = ""
    # audio_file_path = "notes1.mp3"
    # file_path = "content/drive/MyDrive/Colab_Notebooks/artifacts/clinical_notes/code_NA/Note 1.pdf"

    file_path = "content/drive/MyDrive/Colab_Notebooks/artifacts/clinical_notes/code_NA/Note 3.txt"

    documents = load_file(file_path)
    # transcript = mp3_to_text(audio_file_path)

    # Step 2: Get the ICD-10 and CPT codes
    codes_table = get_codes(query)
    print("\nICD-10 and CPT Codes Table:")

    [print(line) for line in codes_table]


if __name__ == "__main__":
    main()